In [107]:
import pandas as pd

vessel_types_dict = {
	'Cargo ship': 70,
	'Fishing vessel': 30,
	'Tanker': 80
	}
navstatus_dict =    {
    "AIS-SART is active": 0,
    "Aground": 2,
    "At anchor": 1,
    "Constrained by her draught": 3,
    "Engaged in Fishing": 4,
    "Moored": 5,
    "Not defined": 6,
    "Not under command": 7,
    "Reserved for future amendment of Navigational Status for HSC": 8,
    "Reserved for future amendment of Navigational Status for WIG": 9,
    "Reserved for future use": 10,
    "Restricted manoeuverability": 11,
    "Under way sailing": 12,
    "Under way using engine": 13,
    "aground": 2,
    "at anchor": 1,
    "constrained by draught": 3,
    "engaged in fishing": 4,
    "moored": 5,
    "not available": 6,
    "not under command": 7,
    "reserved for future amendment of navigational status for HSC": 8,
    "reserved for future amendment of navigational status for WIG": 9,
    "reserved for future use": 10,
    "restricted manoeuvrability": 11,
    "under way using engine": 13,
    "underway sailing": 12,
}

In [119]:
csv = 'data/central_med/raw_data.csv'
df = pd.read_csv(csv)
df["timestamp"] = pd.to_datetime(df['timestamp'])
df['timestamp'] = df['timestamp'].astype(int) // 10**9  
df.dropna(subset=['vesseltype'], axis=0, inplace=True)
df['rot'] = 0
df["vessel_type_int"] = 0
df['vessel_type_int'] = df['vesseltype'].map(vessel_types_dict)
mask = (df["vessel_type_int"] > 0)
df = df[mask]
df['navstatus_int'] = df['navstatus'].map(navstatus_dict)
df["vessel_type_int"] = df.vessel_type_int.astype('int')
df.drop(["vesseltype", "vesselclass", "navstatus"], axis=1, inplace=True)


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1603080 entries, 0 to 3087035
Data columns (total 10 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   lat              1603080 non-null  float64
 1   lon              1603080 non-null  float64
 2   speed            1603080 non-null  int64  
 3   course           1603080 non-null  int64  
 4   heading          1603080 non-null  int64  
 5   timestamp        1603080 non-null  int64  
 6   mmsi             1603080 non-null  int64  
 7   rot              1603080 non-null  int64  
 8   vessel_type_int  1603080 non-null  int64  
 9   navstatus_int    1603080 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 134.5 MB


In [125]:
df.to_csv('data/central_med/converted_raw_data.csv',sep=';')

In [157]:
import pickle

# Specify the path to your pickle file
pickle_file_path = 'data/ct_2017010203_10_20/ct_2017010203_10_20_train.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as file:
    # Load the content from the pickle file
    content = pickle.load(file)

In [159]:
content[0][0]

array([6.90930000e-01, 7.21915000e-01, 3.00000000e-01, 6.48888889e-01,
       2.30000000e+02, 0.00000000e+00, 0.00000000e+00, 1.48322880e+09,
       3.04655000e+08])

In [163]:
pickle_file_path = 'data/central_med/ct_centralmed_valid_track.pkl'

# Open the pickle file in binary mode
with open(pickle_file_path, 'rb') as file:
    # Load the content from the pickle file
    content = pickle.load(file)

In [165]:
content.keys()

dict_keys([305864000, 255805612, 255805611, 211879870, 247149100, 249693000, 477108100, 209003000, 538008428, 255806148, 431048000, 255806263, 311000080, 373765000, 273450990, 209570000, 636020980, 636021310, 271000973, 352001685, 256128000, 566409000, 304461000, 249320000, 215330000, 248230000, 210221000, 636018321, 209720000, 371039000, 255805558, 636093155, 249677000, 636092253, 248047000, 370273000, 255805571, 256445000, 538008717, 672708000, 352978218, 352898773, 215199000, 538007504, 636014251, 312561000, 636093089, 255806440, 636020585, 372961000, 538007812, 563140700, 258987000, 672704000, 229730000, 356811000, 256901000, 538008516, 215233000, 248315000, 353209000, 636018205, 538003617, 256132000, 636016722, 249891000, 247288900, 636020981, 373841000, 352001250, 538009709, 636018194, 247292200, 636019634, 255806063, 277551000, 255805834, 518998763, 564079000, 314652000, 255806056, 636014354, 352001243, 220624000, 636020898, 246399000, 244860000, 351732000, 247318300, 374050000,

In [167]:
content[255805612][0]

array([3.76783330e+01, 9.63333300e+00, 9.00000000e+00, 1.03000000e+02,
       1.04000000e+02, 0.00000000e+00, 1.30000000e+01, 1.70073983e+09,
       2.55805612e+08])

In [102]:
from datetime import datetime
print("Time min: ",datetime.utcfromtimestamp(1706066955).strftime('%Y-%m-%d %H:%M:%SZ'))

Time min:  2024-01-24 03:29:15Z


In [139]:
from tqdm import tqdm
import numpy as np

message_cols = [
        "lat",
        "lon",
        "speed",
        "course",
        "heading",
        "rot",
        "navstatus_int",
        "timestamp",
        "mmsi",
    ]
grouped_df = df.groupby('mmsi')
Vs_train = {
    key: np.array(group.sort_values(by="timestamp")[message_cols])
    for key, group in tqdm(grouped_df)
}

Vs_train = {
    key: np.array(sorted(value, key=lambda m_entry: m_entry[7]))
    for key, value in tqdm(Vs_train.items())
}

  0%|          | 0/4608 [00:00<?, ?it/s]

100%|██████████| 4608/4608 [00:01<00:00, 3722.44it/s]


In [141]:
Vs_train[]

KeyError: 1

In [132]:
VesselTypes[2187]

30